# Pretraining Models

TODO

## Pretraining the Graph-Based Transformer

TODO

In [1]:
import pandas as pd

N_PROC = 12
MAX_MOLS = int(1e6)

In [2]:
smiles = pd.read_table('jupyter/data/chembl_30_ALL.smi', sep='\t', header=0, usecols=('Smiles',)).squeeze('columns').sample(MAX_MOLS)
smiles.shape

(1000000,)

In [3]:
from utils import initLogger

initLogger('pretraining_graph.log')

/home/martin/miniconda3/envs/drugex/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from drugex.data.processing import Standardization

standardizer = Standardization(n_proc=N_PROC)
smiles = standardizer.apply(smiles)
len(smiles)

Standardizing molecules (batch processing): 100%|██████████| 21/21 [03:28<00:00,  9.93s/it]


1000000

In [1]:
from drugex.data.datasets import GraphFragDataSet
import os

graph_input_folder = "data/sets/graph/pretraining"
if not os.path.exists(graph_input_folder):
    os.makedirs(graph_input_folder)

train = GraphFragDataSet(f"{graph_input_folder}/chembl30_train.tsv", rewrite=False)
test = GraphFragDataSet(f"{graph_input_folder}/chembl30_test.tsv", rewrite=False)

/home/martin/miniconda3/envs/drugex/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from drugex.data.fragments import FragmentCorpusEncoder
from drugex.data.fragments import GraphFragmentEncoder, FragmentPairsSplitter
from drugex.molecules.converters.fragmenters import Fragmenter
from drugex.data.corpus.vocabulary import VocGraph

vocabulary = VocGraph(n_frags=4)
encoder = FragmentCorpusEncoder(
    fragmenter=Fragmenter(4, 4, 'brics'),
    encoder=GraphFragmentEncoder(
        vocabulary
    ),
    pairs_splitter=FragmentPairsSplitter(0.1, 1000, make_unique=False),
    n_proc=N_PROC,
    chunk_size=500
)

encoder.apply(smiles, encodingCollectors=[test, train])

Creating fragment-molecule pairs (batch processing): 100%|██████████| 42/42 [22:48<00:00, 32.59s/it]
Encoding fragment-molecule pairs. (batch processing): 100%|██████████| 1/1 [00:02<00:00,  2.21s/it]
Encoding fragment-molecule pairs. (batch processing): 100%|██████████| 277/277 [33:17<00:00,  7.21s/it]


In [ ]:
from drugex.training.models.transform.gpt2graph import GraphModel
from drugex.data.corpus.vocabulary import VocGraph
from utils import SimpleMonitor

vocabulary = VocGraph(n_frags=4)
model = GraphModel(voc_trg=vocabulary, use_gpus=(3,4,5))
print(model.gpus)
monitor = SimpleMonitor('data/models/pretrained/graph/chembl_pretrained')
model.fit(test.asDataLoader(512), train.asDataLoader(512), monitor=monitor)

(3, 4, 5)


  2%|▏         | 2/100 [5:03:20<247:40:07, 9098.04s/it]